# The Three Pillars - Instruction, Context, and Constraints

Welcome! This notebook brings our most important prompt design framework to life: **The Three Pillars**. As we discussed, nearly every effective prompt is a combination of these three components:

1. **Instruction:** What you want the model to *do*. (e.g., "Write unit tests.")
2. **Context:** The information the model *needs* to perform the instruction. (e.g., The code to be tested.)
3. **Constraints:** The *rules* the output must follow. (e.g., "Use the `unittest` framework" or "Format as JSON.")

To see this in action, we'll perform a common developer task: **generating unit tests for a Python function.** We will build our prompt iteratively, starting with just an instruction and adding pillars one by one to see how the quality of the output dramatically improves.

## The Function to be Tested (Our Context)

First, let's define the code we want to test. This Python function will serve as the **Context** for our prompt later on. It's designed to find the longest string in a list and includes a few edge cases we'll want to test.

In [ ]:
# This is the function we will ask the LLM to write tests for.

def find_longest_string(strings: list[str]) -> str | None:
    """
    Given a list of strings, finds and returns the longest string.

    - If the list is empty, it returns None.
    - If there's a tie, it returns the first one found.
    - It raises a TypeError if the list contains non-string elements.
    """
    if not strings:
        return None

    longest_string = ""
    for s in strings:
        if not isinstance(s, str):
            raise TypeError("All elements in the list must be strings.")
        if len(s) > len(longest_string):
            longest_string = s

    return longest_string

In [ ]:
import inspect
function_source_code = inspect.getsource(find_longest_string)

print("--- Function to be tested ---")
print(function_source_code)

## Setup: Reusable Helper Functions

In [ ]:
import os
import litellm
from dotenv import load_dotenv
from textwrap import dedent

load_dotenv()

MODEL_NAME = "openai/gpt-4o-mini"
MAX_TOKENS_DEFAULT = 200

def get_completion(
    prompt,
    model=MODEL_NAME,
    max_tokens=MAX_TOKENS_DEFAULT,
    **kwargs
):
    parsed_messages = []

    if type(prompt) is str:
        parsed_messages = [
            {
                "role": "user",
                "content": prompt
            }
        ]
    else:
        parsed_messages = prompt

    response = litellm.completion(
        model=model,
        messages=parsed_messages,
        max_tokens=max_tokens,
        **kwargs
    )

    return response.choices[0].message.content

print("Helper functions are defined and ready.")

## Attempt 1: Using Only an Instruction (Pillar 1)

Let's start with the most basic prompt possible: just an instruction. We'll ask the model to perform a task without giving it any of the necessary information.

As you'll see, the model can't read our minds. Without context, the instruction is ambiguous and the model has to guess, often resulting in a generic, unhelpful, or incomplete response.

## Attempt 2: Adding Context (Pillars 1 & 2)

Now, let's provide the second pillar: **Context**. We'll give the model the same instruction, but this time we'll also provide the source code of the function we want it to test.

This is a huge improvement. The model now has the necessary information to perform the task. However, as we'll see, without constraints, the output might not be exactly what we need for our project.

## Attempt 3: Adding Constraints for Precision (Pillars 1, 2, & 3)

This is the final and most powerful step. We will add the third pillar: **Constraints**. We will provide the same instruction and context, but now we'll add a specific set of rules that the output must follow. This gives us precise control over the final result.